In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# Crear Espectrogramas

En este notebook de python se van a generar los espectrogramas de mel que posteriormente entrenaran a la red neruronal.

La siguiente función crea los espectrogramas de todas las canciones en una carpeta.
Parametros:

    -input_path: La carpeta que contiene las canciones
    -segment_duration: Duración en segundos de los espectrogramas

In [ ]:

mel_song = []
def create_spectrograms(input_path,segment_duration):
    for file in os.listdir(input_path):
        last_folder = os.path.basename(os.path.basename(input_path))

        audio_file = os.path.join(input_path, file)
        try:
            y, sr = librosa.load(audio_file)
        except:
            print(f"Error loading {audio_file}")
            pass
       
        start_sec = 0
        duration = len(y)/sr
        for i in range(0,int(duration/segment_duration)):
            fig =  plt.figure(figsize=(216/100, 224/100))  

            ax = fig.add_subplot(1, 1, 1)
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            mel = librosa.feature.melspectrogram(y=y[int(start_sec*sr):int((start_sec+segment_duration)*sr)], sr=sr, n_mels=224)
            start_sec += segment_duration
            mel = librosa.power_to_db(mel)
            librosa.display.specshow(mel,sr=sr,cmap='magma')
            # Define the path
            path = f'C:\Diego\music-genre-classifier\imagenes\gtzan2\{last_folder}'
            # Check if the path exists
            if not os.path.exists(path):
                # Create the path
                os.makedirs(path)
            plt.savefig(f'{path}/{file}_{i}.png')
   
            plt.clf()

import os

path = "C:/Diego/music-genre-classifier/audio/gtzan"

directorios = [nombre for nombre in os.listdir(path) if os.path.isdir(os.path.join(path, nombre))]

for dir in directorios:
    create_spectrograms(f'{path}/{dir}',5)

# Audio augmentation
En este notebook implementara una función que aplicara data augmentation a los audios, con el objetivo de aumentar la diversidad en las muestras. Y en consecuencia tener una red neuronal más sólida. 

In [ ]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def create_spectrograms(audio, sr, segment_duration, output_path, filename_prefix):
    start_sec = 0
    duration = len(audio) / sr
    for i in range(0, int(duration / segment_duration)):
        fig = plt.figure(figsize=(216 / 100, 224 / 100))  # Tamaño en pulgadas

        ax = fig.add_subplot(1, 1, 1)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
        mel = librosa.feature.melspectrogram(
            y=audio[int(start_sec * sr):int((start_sec + segment_duration) * sr)],
            sr=sr, n_mels=224
        )
        start_sec += segment_duration
        mel = librosa.power_to_db(mel)
        librosa.display.specshow(mel, sr=sr, cmap='magma')

        # Define the path
        path = output_path
        # Check if the path exists
        if not os.path.exists(path):
            # Create the path
            os.makedirs(path)
        plt.savefig(f'{path}/{filename_prefix}_{i}.png')
        plt.clf()
        plt.close()

def apply_audio_augmentations(folder_path, dir):
    augment = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        TimeStretch(min_rate=0.8, max_rate=1.1, p=0.5),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
    ])
    
    for filename in os.listdir(folder_path):
        song_path = os.path.join(folder_path, filename)
        try:
            # Load the audio file
            audio, sr = librosa.load(song_path)
            
            # Apply audio augmentations
            augmented_audio = augment(samples=audio, sample_rate=sr)
            
            # Create spectrograms for augmented audio
            create_spectrograms(augmented_audio, sr, 5, f'C:/Diego/music-genre-classifier/imagenes/gtzan2/{dir}', f'{filename}_augmented')
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

# Base directory where audio files are stored
path = "C:/Diego/music-genre-classifier/audio/gtzan"

# List all subdirectories (genres)
directorios = [nombre for nombre in os.listdir(path) if os.path.isdir(os.path.join(path, nombre))]

# Generate spectrograms for each audio file in each subdirectory after applying augmentations
for dir in directorios:
    apply_audio_augmentations(f'{path}/{dir}', dir)


In [ ]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift

def apply_audio_augmentations(folder_path, dir):
    # Aumentaciones de audio a aplicar
    augment = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),  # Añadir ruido gaussiano al audio 
        TimeStretch(min_rate=0.8, max_rate=1.1, p=0.5),  # Estirar o comprimir el tiempo del audio 
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5)  # Cambiar el tono del audio en un rango de -4 a 4 semitonos 
    ])
    
    # Iterar sobre cada archivo en el directorio especificado por folder_path
    for filename in os.listdir(folder_path):
        song_path = os.path.join(folder_path, filename)  # Obtener la ruta completa del archivo de audio
        try:
            # Cargar el archivo de audio
            audio, sr = librosa.load(song_path)
            
            # Aplicar las aumentaciones de audio
            augmented_audio = augment(samples=audio, sample_rate=sr)
            
            # Crear espectrogramas para el audio aumentado y guardarlos en la ruta especificada
            create_spectrograms(augmented_audio, sr, 5, f'C:/Diego/music-genre-classifier/imagenes/gtzan2/{dir}', f'{filename}_augmented')
        except Exception as e:
            # Manejar excepciones e imprimir cualquier error que ocurra durante el procesamiento
            print(f"Error processing {filename}: {str(e)}")
